In [2]:
from datetime import datetime
import re

In [104]:
# with open(r"crawl_public_group.log") as f:
#     logs_data = [line.strip() for line in f.readlines()]
test_logs = """[2021-07-28 21:08:07,642]:[INFO]:Getting firefox driver
[2021-07-28 21:08:10,668]:[INFO]:Login ...
[2021-07-28 21:08:23,592]:[INFO]:User: 522164805, Password: matkhau2213
[2021-07-28 21:08:23,592]:[INFO]:Crawling group 1189517794437569
[2021-07-28 21:08:23,592]:[INFO]:Go into group ...
[2021-07-28 21:08:34,003]:[INFO]:Scrolling page ...
[2021-07-28 21:08:34,003]:[INFO]:0%|          | 0/10 [00:00<?, ?it/s]
[2021-07-28 21:08:37,022]:[INFO]:10%|#         | 1/10 [00:03<00:27,  3.02s/it]
[2021-07-28 21:08:40,037]:[INFO]:20%|##        | 2/10 [00:06<00:24,  3.02s/it]
[2021-07-28 21:08:43,054]:[INFO]:30%|###       | 3/10 [00:09<00:21,  3.02s/it]
[2021-07-28 21:08:46,069]:[INFO]:40%|####      | 4/10 [00:12<00:18,  3.02s/it]
[2021-07-28 21:08:49,083]:[INFO]:50%|#####     | 5/10 [00:15<00:15,  3.02s/it]
[2021-07-28 21:08:52,089]:[INFO]:60%|######    | 6/10 [00:18<00:12,  3.01s/it]
[2021-07-28 21:08:55,103]:[INFO]:70%|#######   | 7/10 [00:21<00:09,  3.01s/it]
[2021-07-28 21:08:58,117]:[INFO]:80%|########  | 8/10 [00:24<00:06,  3.01s/it]
[2021-07-28 21:09:01,132]:[INFO]:90%|######### | 9/10 [00:27<00:03,  3.01s/it]
[2021-07-28 21:09:04,145]:[INFO]:100%|##########| 10/10 [00:30<00:00,  3.01s/it]
[2021-07-28 21:09:04,145]:[INFO]:100%|##########| 10/10 [00:30<00:00,  3.01s/it]
[2021-07-28 21:09:04,164]:[INFO]:Gathering post urls ...
[2021-07-28 21:09:05,209]:[INFO]:Got 181 posts url
[2021-07-28 21:09:05,209]:[INFO]:Gathering post data  ...
[2021-07-28 21:09:11,355]:[INFO]:ID: 1189517794437569_4314844601904857
[2021-07-28 21:09:11,366]:[INFO]:Num like: 1
[2021-07-28 21:09:15,551]:[INFO]:Extracted 1 uids liked
[2021-07-28 21:09:18,555]:[INFO]:Remaining: 180
[2021-07-28 21:09:24,588]:[INFO]:ID: 1189517794437569_4314558845266766
[2021-07-28 21:09:24,594]:[INFO]:Num like: 1
[2021-07-28 21:09:29,460]:[INFO]:Extracted 1 uids liked"""
logs_data = [line for line in test_logs.split("\n")]

In [108]:
def _extract_usr_pwd(text):
    usr_pattern = re.compile(r"(?<=User: ).+(?=,)")
    pwd_pattern = re.compile(r"(?<=Password: ).+")
    try:
        usr = re.findall(usr_pattern, text)[0]
        pwd = re.findall(pwd_pattern, text)[0]
        return {usr:pwd}
    except:
        return None
def _extract_group_id(text):
    group_id_pattern = re.compile(r"(?<=group )\d+")
    try:
        return re.findall(group_id_pattern, text)[0]
    except:
        return None
def _extract_time(text):
    time_pattern = re.compile(r"\[.+\]")
    try:
        contains_time = re.match(time_pattern, text)
        if contains_time:
        #     return datetime.strptime(text[1:20], r'%Y-%m-%d %H:%M:%S')
            return text[1:20]
    except:
        return None
def _extract_post_id(text):
    return text[36:]

group_id = None
curent_time = None
account = None
post_id = None
error = None

def _reset_values():
    group_id = None
    curent_time = None
    account = None
    post_id = None
    error = None

post_list = []

for line in logs_data:
    if line == "=======":
        _reset_values()

    current_time = _extract_time(line)
    if "[INFO]:User" in line:
        account = _extract_usr_pwd(line)
    elif "Crawling group" in line:
        group_id = _extract_group_id(line)
    elif "[INFO]:ID:" in line:
        post_id = _extract_post_id(line)
        post_list.append({
            "post_id": post_id,
            "account": account,
            "group_id": group_id,
            "error": "None",
            "time":current_time
        })
    elif "[ERROR]:Error get id" in line:
        post_id = "None"
        error = "Error get id"
        post_list.append({
            "post_id": post_id,
            "account": account,
            "group_id": group_id,
            "error": error,
            "time":current_time
        })
    elif "Message: Unable to locate element: ._5rgr.async_like" in line:
        post_id = "None"
        error = "Unable to read posts"
        post_list.append({
            "post_id": post_id,
            "account": account,
            "group_id": group_id,
            "error": error,
            "time":current_time
        })

In [109]:
nearest_log_time = datetime.strptime(post_list[-1]['time'], r'%Y-%m-%d %H:%M:%S')
one_day_log = []
for post in post_list[::-1]:
    one_day_log.append(post)
    log_time = datetime.strptime(post['time'], r'%Y-%m-%d %H:%M:%S')
    day_diff = (nearest_log_time - log_time).total_seconds()/60/60/24
    if day_diff > 1:
        break

In [115]:
post_per_day = 0
error_post_per_day = 0
group_info = {}
group_error_info = {}
accounts_info = {}
account_error_info = {}
for post in one_day_log:
    #calculate number of posts per day
    if post['post_id'] != "None":
        post_per_day+=1
    else:
        error_post_per_day += 1
    #calculate number of posts in each group
    if post['group_id'] not in group_info:
        if post['post_id'] != "None":
            group_info[post['group_id']] = 1
            group_error_info[post['group_id']] = 0
        else:
            group_info[post['group_id']] = 0
            group_error_info[post['group_id']] = 1
    else: 
        if post['post_id'] != "None":
            group_info[post['group_id']] += 1
        else:
            group_error_info[post['group_id']] += 1
    #calculate number of posts in each account
    username = list(post['account'].keys())[0]
    if username not in accounts_info:
        if post['post_id'] != "None":
            accounts_info[username] = 1
            account_error_info[username] = 0
        else:
            accounts_info[username] = 0
            account_error_info[username] = 1
    else: 
        if post['post_id'] != "None":
            accounts_info[username] += 1
        else:
            account_error_info[username] += 1


In [116]:
print(f"Number of post per day: {post_per_day}")
print(f"Number of error post per day: {error_post_per_day}")
print(f"Number of post in each group:\n{group_info}")
print(f"Number of error post in each group:\n{group_error_info}")
print(f"Number of post in each account:\n{accounts_info}")
print(f"Number of error post in each account:\n{account_error_info}")

Number of post per day: 2
Number of error post per day: 0
Number of post in each group:
{'1189517794437569': 2}
Number of error post in each group:
{'1189517794437569': 0}
Number of post in each account:
{'522164805': 2}
Number of error post in each account:
{'522164805': 0}


In [101]:
sum = 0
for key, value in group_error_info.items():
    sum += value
print(sum)

0


In [1]:
import json

with open("wow.json", "r") as f:
    data = json.load(f)

In [5]:
x = """{"create":{"_index":"test2"}}
{'post_id': ' 1189517794437569_4314844601904857', 'account': {'522164805': 'matkhau2213'}, 'group_id': '1189517794437569', 'error': 'None', 'time': '2021-07-28 21:09:11'}
{"create":{"_index":"test2"}}
{'post_id': ' 1189517794437569_4314558845266766', 'account': {'522164805': 'matkhau2213'}, 'group_id': '1189517794437569', 'error': 'None', 'time': '2021-07-28 21:09:24'}
{"create":{"_index":"test2"}}
{'post_id': ' 1189517794437569_4314628605259790', 'account': {'522164805': 'matkhau2213'}, 'group_id': '1189517794437569', 'error': 'None', 'time': '2021-07-28 21:09:38'}
{"create":{"_index":"test2"}}
{'post_id': ' 1189517794437569_4317463654976285', 'account': {'522164805': 'matkhau2213'}, 'group_id': '1189517794437569', 'error': 'None', 'time': '2021-07-28 21:09:45'}
{"create":{"_index":"test2"}}
{'post_id': ' 1189517794437569_4314807165241934', 'account': {'522164805': 'matkhau2213'}, 'group_id': '1189517794437569', 'error': 'None', 'time': '2021-07-28 21:09:56'}
{"create":{"_index":"test2"}}
{'post_id': ' 1189517794437569_4316067341782583', 'account': {'522164805': 'matkhau2213'}, 'group_id': '1189517794437569', 'error': 'None', 'time': '2021-07-28 21:10:05'}
{"create":{"_index":"test2"}}
{'post_id': ' 1189517794437569_4318434674879183', 'account': {'522164805': 'matkhau2213'}, 'group_id': '1189517794437569', 'error': 'None', 'time': '2021-07-28 21:10:16'}
{"create":{"_index":"test2"}}
{'post_id': ' 1189517794437569_4314765105246140', 'account': {'522164805': 'matkhau2213'}, 'group_id': '1189517794437569', 'error': 'None', 'time': '2021-07-28 21:10:29'}
{"create":{"_index":"test2"}}
{'post_id': ' 1189517794437569_4313572782032039', 'account': {'522164805': 'matkhau2213'}, 'group_id': '1189517794437569', 'error': 'None', 'time': '2021-07-28 21:10:36'}
{"create":{"_index":"test2"}}
{'post_id': ' 101656875072560_4312027895519861', 'account': {'522164805': 'matkhau2213'}, 'group_id': '1189517794437569', 'error': 'None', 'time': '2021-07-28 21:10:54'}
{"create":{"_index":"test2"}}
{'post_id': ' 1189517794437569_4315245791864738', 'account': {'522164805': 'matkhau2213'}, 'group_id': '1189517794437569', 'error': 'None', 'time': '2021-07-28 21:11:02'}
{"create":{"_index":"test2"}}
{'post_id': ' 1189517794437569_4313243318731652', 'account': {'522164805': 'matkhau2213'}, 'group_id': '1189517794437569', 'error': 'None', 'time': '2021-07-28 21:11:13'}
{"create":{"_index":"test2"}}"""
x = x.replace("'", '"')
print(x)

{"create":{"_index":"test2"}}
{"post_id": " 1189517794437569_4314844601904857", "account": {"522164805": "matkhau2213"}, "group_id": "1189517794437569", "error": "None", "time": "2021-07-28 21:09:11"}
{"create":{"_index":"test2"}}
{"post_id": " 1189517794437569_4314558845266766", "account": {"522164805": "matkhau2213"}, "group_id": "1189517794437569", "error": "None", "time": "2021-07-28 21:09:24"}
{"create":{"_index":"test2"}}
{"post_id": " 1189517794437569_4314628605259790", "account": {"522164805": "matkhau2213"}, "group_id": "1189517794437569", "error": "None", "time": "2021-07-28 21:09:38"}
{"create":{"_index":"test2"}}
{"post_id": " 1189517794437569_4317463654976285", "account": {"522164805": "matkhau2213"}, "group_id": "1189517794437569", "error": "None", "time": "2021-07-28 21:09:45"}
{"create":{"_index":"test2"}}
{"post_id": " 1189517794437569_4314807165241934", "account": {"522164805": "matkhau2213"}, "group_id": "1189517794437569", "error": "None", "time": "2021-07-28 21:09:

In [4]:
for post in data:
    print('{"create":{"_index":"test2"}}')
    print(post)

{"create":{"_index":"test2"}}
{'post_id': ' 1189517794437569_4314844601904857', 'account': {'522164805': 'matkhau2213'}, 'group_id': '1189517794437569', 'error': 'None', 'time': '2021-07-28 21:09:11'}
{"create":{"_index":"test2"}}
{'post_id': ' 1189517794437569_4314558845266766', 'account': {'522164805': 'matkhau2213'}, 'group_id': '1189517794437569', 'error': 'None', 'time': '2021-07-28 21:09:24'}
{"create":{"_index":"test2"}}
{'post_id': ' 1189517794437569_4314628605259790', 'account': {'522164805': 'matkhau2213'}, 'group_id': '1189517794437569', 'error': 'None', 'time': '2021-07-28 21:09:38'}
{"create":{"_index":"test2"}}
{'post_id': ' 1189517794437569_4317463654976285', 'account': {'522164805': 'matkhau2213'}, 'group_id': '1189517794437569', 'error': 'None', 'time': '2021-07-28 21:09:45'}
{"create":{"_index":"test2"}}
{'post_id': ' 1189517794437569_4314807165241934', 'account': {'522164805': 'matkhau2213'}, 'group_id': '1189517794437569', 'error': 'None', 'time': '2021-07-28 21:09: